# Exploratory Data Analysis


## Import necessary packages


In [16]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") 
import pandas as pd
from src.data.data_ingestion import DataIngestion
from src.data.data_cleaning import DataCleaning

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Ingest raw data

In [24]:
raw_data = DataIngestion().ingest()

[2023-08-14 20:19:36,692] 37 src.utils.logger - INFO - Started data ingestion process...
[2023-08-14 20:19:36,694] 40 src.utils.logger - INFO - Data already ingested to CSV: /home/umar/Workspace/personal/msc-dissertation/src/data/../../data/raw/raw.csv


## Read raw data into a data frame

In [25]:
df = pd.read_csv(raw_data)
df.head()

/tmp/ipykernel_2390/2507576894.py:1: DtypeWarning: Columns (9,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_data)


,ID,Jira_ID,Issue_Key,URL,Title,Description,Description_Text,Description_Code,Type,Priority,...,Description_Changed_After_Estimation,Story_Point_Changed_After_Estimation,Pull_Request_URL,Creator_ID,Reporter_ID,Assignee_ID,Project_ID,Sprint_ID,Repository_Name,Poject_Name
0,65,77638,XD-3768,https://jira.spring.io/rest/api/2/issue/77638,"""How do I make a job restartable in spring xd""","""The jobs that appear under Executions section...","""""""The jobs that appear under Executions secti...",NaN,Bug,Major,...,0,0,NaN,68.0,68.0,NaN,1,NaN,Spring,Spring XD
1,66,77511,XD-3767,https://jira.spring.io/rest/api/2/issue/77511,"""admin config timezone command does not work""","""Working with Spring-XD version 1.3.2.RELEASE ...","""""""Working with Spring-XD version 1.3.2.RELEAS...",""" xd:>admin config admin config info ad...",Bug,Trivial,...,0,0,NaN,69.0,69.0,NaN,1,NaN,Spring,Spring XD
2,67,77130,XD-3766,https://jira.spring.io/rest/api/2/issue/77130,"""Module Upload command not pushing jar to all ...","""My project 7 node cluster and in that 2 node ...","""""""My project 7 node cluster and in that 2 nod...",NaN,Bug,Major,...,0,0,NaN,70.0,70.0,NaN,1,NaN,Spring,Spring XD
3,68,71950,XD-3765,https://jira.spring.io/rest/api/2/issue/71950,"""Fix stream failover ""","""See https://github.com/spring-projects/spring...","""""""See https://github.com/spring-projects/spri...",NaN,Story,Minor,...,0,0,NaN,72.0,72.0,71.0,1,NaN,Spring,Spring XD
4,69,71805,XD-3764,https://jira.spring.io/rest/api/2/issue/71805,"""SpringXD Job is still executing even after fo...","""I'm trying to run a Job on SpringXD and the j...","""""""I'm trying to run a Job on SpringXD and the...",NaN,Bug,Major,...,0,0,NaN,73.0,73.0,NaN,1,NaN,Spring,Spring XD


In [26]:
df.describe()

,ID,Jira_ID,Story_Point,Timespent,In_Progress_Minutes,Total_Effort_Minutes,Resolution_Time_Minutes,Title_Changed_After_Estimation,Description_Changed_After_Estimation,Story_Point_Changed_After_Estimation,Creator_ID,Reporter_ID,Assignee_ID,Project_ID,Sprint_ID
count,458232.000000,4.582320e+05,6.542700e+04,10243.000000,4.582320e+05,4.582320e+05,4.582320e+05,458232.000000,458232.0,458232.000000,456641.000000,455736.000000,261831.000000,458232.000000,43475.000000
mean,250427.756628,6.842263e+05,4.135718e+02,323.090501,5.238023e+03,2.343935e+04,4.084872e+05,0.141330,0.0,0.020068,120547.190712,120825.496105,133827.102066,24.998507,2711.075791
std,154217.334318,2.121512e+06,1.044232e+05,761.792096,5.792574e+04,1.286546e+05,8.887023e+05,0.348362,0.0,0.140235,62373.774892,62234.714879,60001.012434,10.805999,1512.380726
min,65.000000,1.000000e+04,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0,0.000000,68.000000,68.000000,71.000000,1.000000,1.000000
25%,114626.750000,5.834075e+04,1.000000e+00,46.000000,0.000000e+00,0.000000e+00,4.000000e+00,0.000000,0.0,0.000000,73529.000000,74305.000000,116063.000000,18.000000,1395.500000
50%,229184.500000,1.471675e+05,3.000000e+00,120.000000,0.000000e+00,0.000000e+00,1.535700e+04,0.000000,0.0,0.000000,145748.000000,145748.000000,151325.000000,23.000000,2941.000000
75%,394473.250000,6.167282e+05,5.000000e+00,300.000000,0.000000e+00,1.458000e+03,2.634485e+05,0.000000,0.0,0.000000,164408.000000,164408.000000,166050.000000,34.000000,4053.000000
max,509035.000000,1.333610e+07,2.671011e+07,24622.000000,4.771423e+06,4.806555e+06,9.156071e+06,1.000000,0.0,1.000000,209125.000000,209125.000000,208978.000000,44.000000,5027.000000


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458232 entries, 0 to 458231
Data columns (total 32 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   ID                                    458232 non-null  int64  
 1   Jira_ID                               458232 non-null  int64  
 2   Issue_Key                             458232 non-null  object 
 3   URL                                   458232 non-null  object 
 4   Title                                 458232 non-null  object 
 5   Description                           429104 non-null  object 
 6   Description_Text                      429104 non-null  object 
 7   Description_Code                      59005 non-null   object 
 8   Type                                  458232 non-null  object 
 9   Priority                              331573 non-null  object 
 10  Status                                458232 non-null  object 
 11  

In [28]:
DataCleaning.assess_missing_datax(df)

                  Total Missing  Percent Missing (%)
Pull_Request_URL         457586            99.859023
Timespent                447989            97.764669
Sprint_ID                414757            90.512448
Description_Code         399227            87.123335
Estimation_Date          392805            85.721861
Story_Point              392805            85.721861
Assignee_ID              196401            42.860603
Priority                 126659            27.640802
Resolution_Date          105600            23.045095
Resolution                77498            16.912394
Description               29128             6.356605
Description_Text          29128             6.356605
Reporter_ID                2496             0.544702
Creator_ID                 1591             0.347204


In [21]:
cleaned_df = DataCleaning().clean_data(df)
cleaned_df.head()

KeyError: 'In_Progress_Minutes'

                 Total Missing  Percent Missing (%)
Timespent               447989            97.764669
Estimation_Date         392805            85.721861
Assignee_ID             196401            42.860603
Priority                126659            27.640802
Resolution_Date         105600            23.045095
Resolution               77498            16.912394
Description              29128             6.356605
Reporter_ID               2496             0.544702
Creator_ID                1591             0.347204


,ID,Jira_ID,Issue_Key,Title,Description,Type,Priority,Status,Resolution,Creation_Date,...,Timespent,Resolution_Time_Minutes,Total_Effort_Minutes,Title_Changed_After_Estimation,Description_Changed_After_Estimation,Assignee_ID,Creator_ID,Reporter_ID,Repository_Name,Poject_Name
0,65,77638,XD-3768,"""How do I make a job restartable in spring xd""","""The jobs that appear under Executions section...",Bug,Major,To Do,NaN,2017-07-10 13:41:25,...,NaN,0.0,0.0,0,0,NaN,68.0,68.0,Spring,Spring XD
1,66,77511,XD-3767,"""admin config timezone command does not work""","""Working with Spring-XD version 1.3.2.RELEASE ...",Bug,Trivial,To Do,NaN,2017-06-26 16:26:27,...,NaN,0.0,0.0,0,0,NaN,69.0,69.0,Spring,Spring XD
2,67,77130,XD-3766,"""Module Upload command not pushing jar to all ...","""My project 7 node cluster and in that 2 node ...",Bug,Major,To Do,NaN,2017-05-19 22:28:43,...,NaN,0.0,0.0,0,0,NaN,70.0,70.0,Spring,Spring XD
3,68,71950,XD-3765,"""Fix stream failover ""","""See https://github.com/spring-projects/spring...",Story,Minor,Done,Complete,2017-03-21 16:54:44,...,NaN,0.0,0.0,1,0,71.0,72.0,72.0,Spring,Spring XD
4,69,71805,XD-3764,"""SpringXD Job is still executing even after fo...","""I'm trying to run a Job on SpringXD and the j...",Bug,Major,To Do,NaN,2017-03-06 20:01:41,...,NaN,0.0,0.0,0,0,NaN,73.0,73.0,Spring,Spring XD
